# Clean Job Posting Dates
This notebook (by Rachel) cleans the job posting dates, performs necessary date calculations (in the case of "posted yesterday", "posted 1 week ago", etc.) and updates the database with the clean dates.

In [3]:
import pandas as pd
import numpy as np
import pymysql
from tqdm import tqdm

In [4]:
import db_info as db

In [5]:
connection = pymysql.connect(host=db.DB_HOST,
    user=db.DB_USER,
    password=db.DB_PASSWORD,
    database=db.DB_NAME,
    port=db.DB_PORT,
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor)

cursor = connection.cursor()
cursor.execute("SELECT * FROM jd;")
table = cursor.fetchall()
connection.close()

In [6]:
df = pd.json_normalize(table)

#### Yesterday

In [7]:
df['days_ago'] = np.where(df['post_date'].str.contains('yesterday', case=False), 1, None)

#### Weeks ago

In [8]:
df['days_ago'][df['post_date'].str.contains('weeks', case=False)] = df['post_date'][df['post_date'].str.contains(
    'weeks', case=False)].str.split(' ').str[0].astype(int)*7

<ipython-input-8-5a6fbcc6d4e0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['days_ago'][df['post_date'].str.contains('weeks', case=False)] = df['post_date'][df['post_date'].str.contains(


#### Days ago

In [9]:
df['days_ago'] = np.where(df['post_date'].str.contains('days', case=False) & \
                          df['post_date'].str.contains('posted', case=False),
                          df['post_date'].str.split(' ').str[1],
                          df['days_ago'])

In [10]:
df['days_ago'] = np.where(df['post_date'].str.contains('days', case=False) & \
                          ~df['post_date'].str.contains('posted', case=False),
                          df['post_date'].str.split(' ').str[0],
                          df['days_ago'])

### calculate date posted

In [11]:
df['crawl_timestamp'] = df['crawl_timestamp'].apply(lambda x: x.replace(r' +0000', ''))

In [12]:
df = df[df['crawl_timestamp']!='']

In [13]:
df['crawl_timestamp'] = pd.DatetimeIndex(df['crawl_timestamp'])

In [14]:
df['days_ago'] = df['days_ago'].fillna(0)

In [15]:
df.days_ago = df.days_ago.astype(int)

In [16]:
df['clean_post_date'] = df.apply(lambda x: x['crawl_timestamp'] - pd.Timedelta(days=x['days_ago']), axis=1)

In [17]:
df['clean_post_date'] = np.where(df['days_ago']==0, df['crawl_timestamp'], df['clean_post_date'])

In [18]:
df.groupby(pd.DatetimeIndex(df['clean_post_date']).year)['id'].count()

clean_post_date
2019    9351
2020     123
2021    8510
Name: id, dtype: int64

### Add clean post dates to database

In [26]:
connection = pymysql.connect(host=db.DB_HOST,
    user=db.DB_USER,
    password=db.DB_PASSWORD,
    database=db.DB_NAME,
    port=db.DB_PORT,
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor)
cursor = connection.cursor()
for i in tqdm(range(len(df))):
    curr_id = df.iloc[i]['id']
    curr_clean_date = df.iloc[i]['clean_post_date']
    cursor.execute(f"""UPDATE jd SET clean_post_date='{curr_clean_date}' WHERE id={curr_id};""")
    connection.commit()
connection.close()

100%|██████████| 17984/17984 [43:17<00:00,  6.92it/s] 
